In [12]:
import requests

%load_ext autoreload
%autoreload 2

from notebook import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# # Copie les notebooks et supprime les sorties.
# copy_and_clean_notebooks()

In [3]:
env = AppInsightsAPIEnv(".env")

env.APP_INSIGHTS_API_ID

'7706f68b-7ff7-461c-86e0-f66ad9d4d030'

In [5]:
start_dt = "2021-12-21T10:54:54Z"
end_dt = "2021-12-21T11:54:54Z"

data = get_tn_dialogs(env, start_dt, end_dt)
data.shape

(40, 5)

In [6]:
data.head()

,main_dialog_uuid,timestamp,author,session_id,text
0,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:26.945023+00:00,p10-chatbot-bot,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,What can I help you with today?
1,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:28.479733+00:00,You,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,I want to go to Paris from London tomorrow and...
2,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:29.333668+00:00,p10-chatbot-bot,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,Please confirm the following information:\n- Y...
3,59c2c23c-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:30.250325+00:00,You,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,Non
4,5cdf2384-6253-11ec-a441-72fe038a9524,2021-12-21 11:44:32.158274+00:00,p10-chatbot-bot,lmT29WsWQW9gEr/BwUm9nrvwgvdZ3IcO4Xdx94nMS5g=,What else can I do for you?


In [13]:
analyser = InsatisfactionsAnalyser(data)
analyser.load(PARQUET_PATH, "2021_12_21_analysis")
analyser.display()

In [35]:
analyser.save(PARQUET_PATH, "2021_12_21_analysis")